# Single Perceptron ANN for Iris Dataset

This notebook implements a simple single perceptron artificial neural network to classify iris flowers.

## Perceptron Overview:
- **Single layer neural network** with one output neuron
- Uses **linear activation** with step function for binary classification
- Can only solve **linearly separable** problems
- Simple learning algorithm that updates weights based on errors

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# Load and prepare the iris dataset
def load_iris_data():
    """Load iris dataset for binary classification (Setosa vs Others)"""
    iris = load_iris()
    
    # Create DataFrame
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['species'] = iris.target_names[iris.target]
    
    # Convert to binary classification: Setosa (1) vs Others (0)
    df['is_setosa'] = (iris.target == 0).astype(int)
    
    return df

# Load the data
iris_df = load_iris_data()
print("Iris Dataset Shape:", iris_df.shape)
print("\nFirst 5 rows:")
print(iris_df.head())
print("\nBinary classification target distribution:")
print("Setosa (1):", sum(iris_df['is_setosa'] == 1))
print("Others (0):", sum(iris_df['is_setosa'] == 0))
print("\nOriginal species distribution:")
print(iris_df['species'].value_counts())

In [ ]:
# Simple Single Perceptron Implementation
class SinglePerceptron:
    """Simple Single Perceptron for binary classification"""
    
    def __init__(self, learning_rate=0.1, max_epochs=100):
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.weights = None
        self.bias = None
        self.training_errors = []
    
    def activation_function(self, x):
        """Step function: returns 1 if x >= 0, else 0"""
        return 1 if x >= 0 else 0
    
    def predict_single(self, x):
        """Predict for a single sample"""
        # Calculate weighted sum + bias
        weighted_sum = np.dot(self.weights, x) + self.bias
        # Apply step function
        return self.activation_function(weighted_sum)
    
    def fit(self, X, y):
        """Train the perceptron"""
        n_samples, n_features = X.shape
        
        # Initialize weights and bias randomly (small values)
        self.weights = np.random.uniform(-0.1, 0.1, n_features)
        self.bias = np.random.uniform(-0.1, 0.1)
        
        print(f"Initial weights: {self.weights}")
        print(f"Initial bias: {self.bias:.4f}")
        
        # Training loop
        for epoch in range(self.max_epochs):
            errors = 0
            
            # Process each training sample
            for i in range(n_samples):
                # Forward pass
                prediction = self.predict_single(X[i])
                
                # Calculate error
                error = y[i] - prediction
                
                # Update weights and bias if there's an error
                if error != 0:
                    self.weights += self.learning_rate * error * X[i]
                    self.bias += self.learning_rate * error
                    errors += 1
            
            self.training_errors.append(errors)
            
            # Print progress every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}: {errors} errors")
            
            # Early stopping if no errors
            if errors == 0:
                print(f"\\nConverged at epoch {epoch + 1}!")
                break
        
        print(f"\\nFinal weights: {self.weights}")
        print(f"Final bias: {self.bias:.4f}")
        
        return self
    
    def predict(self, X):
        """Predict for multiple samples"""
        predictions = []
        for i in range(len(X)):
            pred = self.predict_single(X[i])
            predictions.append(pred)
        return np.array(predictions)
    
    def get_decision_boundary_params(self):
        """Get parameters for plotting decision boundary (for 2D visualization)"""
        if len(self.weights) >= 2:
            # For 2D: w1*x1 + w2*x2 + b = 0
            # Solve for x2: x2 = -(w1*x1 + b) / w2
            w1, w2 = self.weights[0], self.weights[1]
            return w1, w2, self.bias
        return None

print("Single Perceptron class implemented successfully!")

In [ ]:
# Prepare data for training
X = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_df['is_setosa']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize the features (important for perceptron)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"\\nTraining set target distribution:")
print(f"Setosa: {sum(y_train)}, Others: {len(y_train) - sum(y_train)}")
print(f"\\nTest set target distribution:")
print(f"Setosa: {sum(y_test)}, Others: {len(y_test) - sum(y_test)}")
print(f"\\nFeatures: {list(X.columns)}")

In [ ]:
# Train the Single Perceptron
print("Training Single Perceptron...")
print("="*50)

# Create and train the perceptron
perceptron = SinglePerceptron(learning_rate=0.1, max_epochs=100)
perceptron.fit(X_train_scaled, y_train.values)

print("\\nTraining completed!")
print("="*50)

In [ ]:
# Make predictions and evaluate
print("Making predictions...")

# Predict on training set
y_train_pred = perceptron.predict(X_train_scaled)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Predict on test set
y_test_pred = perceptron.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\\nPerformance Results:")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

print("\\n" + "="*50)
print("CLASSIFICATION REPORT:")
print("="*50)
print(classification_report(y_test, y_test_pred, target_names=['Others', 'Setosa']))

# Show some example predictions
print("\\n" + "="*50)
print("SAMPLE PREDICTIONS:")
print("="*50)
for i in range(min(10, len(X_test))):
    actual = y_test.iloc[i]
    predicted = y_test_pred[i]
    features = X_test.iloc[i]
    
    print(f"Sample {i+1}:")
    print(f"  Features: Sepal L={features[0]:.2f}, Sepal W={features[1]:.2f}, Petal L={features[2]:.2f}, Petal W={features[3]:.2f}")
    print(f"  Actual: {'Setosa' if actual == 1 else 'Others'}, Predicted: {'Setosa' if predicted == 1 else 'Others'}")
    print(f"  {'✓ Correct' if actual == predicted else '✗ Wrong'}")
    print()

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Training Error Over Epochs
axes[0,0].plot(range(1, len(perceptron.training_errors) + 1), perceptron.training_errors, 'b-o', markersize=4)
axes[0,0].set_title('Training Errors Over Epochs')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Number of Errors')
axes[0,0].grid(True, alpha=0.3)

# 2. Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Others', 'Setosa'], yticklabels=['Others', 'Setosa'],
            ax=axes[0,1])
axes[0,1].set_title('Confusion Matrix')
axes[0,1].set_xlabel('Predicted')
axes[0,1].set_ylabel('Actual')

# 3. Decision Boundary (using first 2 features for visualization)
X_2d = X_train_scaled[:, :2]  # Use first 2 features
y_2d = y_train.values

# Train a simple 2D perceptron for visualization
perceptron_2d = SinglePerceptron(learning_rate=0.1, max_epochs=100)
perceptron_2d.fit(X_2d, y_2d)

# Create mesh grid
h = 0.02
x_min, x_max = X_2d[:, 0].min() - 1, X_2d[:, 0].max() + 1
y_min, y_max = X_2d[:, 1].min() - 1, X_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Make predictions on mesh grid
mesh_points = np.c_[xx.ravel(), yy.ravel()]
Z = perceptron_2d.predict(mesh_points)
Z = Z.reshape(xx.shape)

# Plot decision boundary
axes[1,0].contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)

# Plot training points
setosa_mask = y_2d == 1
others_mask = y_2d == 0

axes[1,0].scatter(X_2d[setosa_mask, 0], X_2d[setosa_mask, 1], 
                 c='red', marker='o', label='Setosa', s=50)
axes[1,0].scatter(X_2d[others_mask, 0], X_2d[others_mask, 1], 
                 c='blue', marker='s', label='Others', s=50)

axes[1,0].set_xlabel('Sepal Length (standardized)')
axes[1,0].set_ylabel('Sepal Width (standardized)')
axes[1,0].set_title('Decision Boundary (2D Projection)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Model Summary and Weights
weights_text = f"""Perceptron Model Summary:

Final Weights:
• Sepal Length: {perceptron.weights[0]:.4f}
• Sepal Width:  {perceptron.weights[1]:.4f}
• Petal Length: {perceptron.weights[2]:.4f}
• Petal Width:  {perceptron.weights[3]:.4f}
• Bias:         {perceptron.bias:.4f}

Performance:
• Training Accuracy: {train_accuracy:.4f}
• Test Accuracy:     {test_accuracy:.4f}
• Epochs to converge: {len(perceptron.training_errors)}

Model Characteristics:
• Linear classifier only
• Can solve linearly separable problems
• Simple learning algorithm
• Fast training"""

axes[1,1].text(0.05, 0.95, weights_text, transform=axes[1,1].transAxes, 
               verticalalignment='top', fontsize=10, fontfamily='monospace',
               bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
axes[1,1].set_title('Model Summary')
axes[1,1].axis('off')

plt.tight_layout()
plt.show()

print(f"\\nFinal Results Summary:")
print(f"{'='*40}")
print(f"Model: Single Perceptron ANN")
print(f"Problem: Binary Classification (Setosa vs Others)")
print(f"Features: 4 iris measurements (standardized)")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Epochs to converge: {len(perceptron.training_errors)}")
print(f"Final weights: {perceptron.weights.round(4)}")
print(f"Final bias: {perceptron.bias:.4f}")
print("✓ Single Perceptron implementation completed successfully!")